# 위키문헌 작품목록 텍스트 스크래핑
- 작성자: [지해인](https://haein.info)

In [3]:
# -*- coding: utf-8 -*-

# 필요한 라이브러리 설치
!pip install mwxml requests tqdm pandas

# 필요한 라이브러리 임포트 (전체 코드에서는 이 셀을 가장 먼저 실행)
import mwxml
import bz2
import json
import os
import re
import urllib.parse
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor, as_completed
import time
from tqdm import tqdm
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from xml.dom import minidom

# -----------------------------------------------------------
# 0. 전역 설정 및 변수
# -----------------------------------------------------------
dump_file = "kowikisource-20251001-pages-articles.xml.bz2"
csv_file = '한국근대소설_TEI_XML_작품목록.csv'
MIN_CONTENT_SIZE_THRESHOLD = 50 # 50바이트 이하인 작품만 덤프 보강 대상으로 선정

print("모든 라이브러리가 로드되었고 변수가 설정되었습니다.")
# (덤프 파일 다운로드 로직은 생략. 파일이 존재한다고 가정)

# -----------------------------------------------------------
# 1. API 및 유틸리티 함수
# -----------------------------------------------------------

# get_categories_from_api, get_year_from_wikidata, enhance_with_api 함수는 기존 코드를 유지합니다.
def get_categories_from_api(page_title):
    try:
        api_url = "https://ko.wikisource.org/w/api.php"
        params = {'action': 'query', 'format': 'json', 'titles': page_title, 'prop': 'categories', 'cllimit': 'max'}
        headers = {'User-Agent': 'WikisourceParser/1.0 (Educational Tutorial)'}
        response = requests.get(api_url, params=params, headers=headers, timeout=10)
        data = response.json()
        pages = data.get('query', {}).get('pages', {})
        page_id = list(pages.keys())[0]
        if page_id == '-1': return []
        categories = pages[page_id].get('categories', [])
        category_names = [cat['title'][3:] for cat in categories if cat['title'].startswith('분류:')]
        return category_names
    except Exception as e:
        return []

def get_year_from_wikidata(page_title):
    try:
        wikisource_api = "https://ko.wikisource.org/w/api.php"
        params = {'action': 'query', 'format': 'json', 'titles': page_title, 'prop': 'pageprops'}
        headers = {'User-Agent': 'WikisourceParser/1.0 (Educational Tutorial)'}
        response = requests.get(wikisource_api, params=params, headers=headers, timeout=10)
        data = response.json()
        pages = data.get('query', {}).get('pages', {})
        page_id = list(pages.keys())[0]
        if page_id == '-1': return None
        wikidata_id = pages[page_id].get('pageprops', {}).get('wikibase_item')
        if not wikidata_id: return None

        wikidata_api = "https://www.wikidata.org/w/api.php"
        params = {'action': 'wbgetentities', 'format': 'json', 'ids': wikidata_id, 'props': 'claims'}
        response = requests.get(wikidata_api, params=params, headers=headers, timeout=10)
        data = response.json()
        claims = data.get('entities', {}).get(wikidata_id, {}).get('claims', {})
        date_properties = ['P577', 'P571', 'P585']

        for prop in date_properties:
            if prop in claims:
                for claim in claims[prop]:
                    try:
                        time_value = claim['mainsnak']['datavalue']['value']['time']
                        year = int(time_value[1:5])
                        if 1800 <= year <= 2030: return year
                    except (KeyError, ValueError):
                        continue
        return None
    except Exception as e:
        return None

def enhance_with_api(page_data):
    title = page_data['title']
    api_categories = get_categories_from_api(title)
    wikidata_year = get_year_from_wikidata(title)
    enhanced = page_data.copy()

    all_categories = list(set(page_data.get('categories', []) + api_categories))
    enhanced['categories'] = all_categories
    enhanced['api_categories'] = api_categories

    year_from_categories = None
    year_categories = [cat for cat in all_categories if '년' in cat and '작품' in cat]
    if year_categories:
        for cat in year_categories:
            year_match = re.search(r'(\d{4})년', cat)
            if year_match:
                year_from_categories = int(year_match.group(1)); break

    enhanced['year'] = year_from_categories or wikidata_year or page_data.get('year')
    enhanced['year_from_categories'] = year_from_categories
    enhanced['year_from_wikidata'] = wikidata_year
    if enhanced['language'] is None: enhanced['language'] = ''
    return enhanced

def extract_metadata(text):
    if not text:
        return {'authors': [], 'categories': [], 'composer': None, 'translator': None,
                'year': None, 'license': None, 'language': None}

    # (메타데이터 추출 로직 생략: 기존 코드 유지)
    authors = []; author_patterns = [r'\[\[저자:([^|\]]+)', r'\|\s*author\s*=\s*\[\[저자:([^|\]]+)']
    for pattern in author_patterns: authors.extend(re.findall(pattern, text))
    categories = re.findall(r'\[\[분류:([^\]]+)\]\]', text)
    composer = None; composer_patterns = [r'([^.]+)\s*작곡', r'작곡가?\s*[:=]\s*([^.\n]+)']
    for pattern in composer_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        if matches: composer = matches[0].strip(); break
    year_matches = re.findall(r'(\d{4})년', text); year = None
    if year_matches: year = max(set(year_matches), key=year_matches.count)
    license_info = None; license_patterns = [r'\{\{(PD-[^}]+)\}\}', r'\{\{(CC-[^}]+)\}\}']
    for pattern in license_patterns:
        matches = re.findall(pattern, text)
        if matches: license_info = matches[0]; break
    language = None
    if '한자' in text and '한글' in text: language = '한자+한글'
    elif '한자' in text: language = '한자'
    elif '한글' in text: language = '한글'
    authors = list(set([a.strip() for a in authors if a.strip()]))
    categories = list(set([c.strip() for c in categories if c.strip()]))

    return {'authors': authors, 'categories': categories, 'composer': composer,
            'translator': None, 'year': year, 'license': license_info, 'language': language}

def clean_text(text):
    """ 위키텍스트에서 마크업을 제거하고 단락 구분자(@@PARAGRAPH@@)를 삽입합니다. """
    if not text: return ""
    cleaned = re.sub(r'\{\{머리말[^{}]*\}\}', '', text, flags=re.DOTALL | re.IGNORECASE)
    cleaned = re.sub(r'\{\{정보[^{}]*\}\}', '', cleaned, flags=re.DOTALL | re.IGNORECASE)
    cleaned = re.sub(r'\{\{[^{}]+\}\}', '', cleaned, flags=re.DOTALL)
    cleaned = re.sub(r'<[^>]+>', '', cleaned)
    cleaned = re.sub(r'\[\[(?:파일|File):[^\]]+\]\]', '', cleaned)
    cleaned = re.sub(r'\[\[분류:[^\]]+\]\]', '', cleaned)
    cleaned = re.sub(r'\n\n+', '@@PARAGRAPH@@', cleaned)
    cleaned = re.sub(r'\[\[[^|\]]*\|([^\]]+)\]\]', r'\1', cleaned)
    cleaned = re.sub(r'\[\[([^\]]+)\]\]', r'\1', cleaned)
    cleaned = re.sub(r"'''?([^']+)'''?", r'\1', cleaned)
    cleaned = re.sub(r'^=+\s*([^=]+)\s*=+$', r'\1', cleaned, flags=re.MULTILINE)
    cleaned = cleaned.replace('\n', ' ')
    cleaned = re.sub(r'\s+', ' ', cleaned)
    return cleaned.strip()

def generate_urls(title, authors):
    base_url = "https://ko.wikisource.org/wiki/"
    page_url = base_url + urllib.parse.quote(title.replace(' ', '_'))
    author_links = []
    for author in authors:
        author_url = base_url + urllib.parse.quote(f"저자:{author}".replace(' ', '_'))
        author_links.append({'name': author, 'url': author_url})
    return page_url, author_links

def extract_page_title(link_url):
    try:
        decoded_url = urllib.parse.unquote(link_url)
        title_with_underscores = decoded_url.split('/wiki/')[1]
        title = title_with_underscores.replace('_', ' ')
        return title
    except:
        return None

def get_wikitext_from_api(page_title):
    try:
        api_url = "https://ko.wikisource.org/w/api.php"
        params = {'action': 'query', 'format': 'json', 'titles': page_title, 'prop': 'revisions',
                  'rvprop': 'content|ids|timestamp|user|size|comment', 'rvslots': 'main', 'rvlimit': '1'}
        headers = {'User-Agent': 'WikisourceParser/1.0 (Educational Tutorial)'}
        response = requests.get(api_url, params=params, headers=headers, timeout=10)
        data = response.json()
        pages = data.get('query', {}).get('pages', {})
        page_id = list(pages.keys())[0]

        if page_id == '-1': return None, None
        revisions = pages[page_id].get('revisions', [])
        if not revisions: return None, None
        revision = revisions[0]
        text = revision.get('slots', {}).get('main', {}).get('*')

        revision_info = {'revision_id': revision.get('revid'), 'last_modified': revision.get('timestamp'),
                         'last_contributor': revision.get('user'), 'size': revision.get('size'), 'comment': revision.get('comment')}
        return text, revision_info

    except Exception as e:
        return None, None

  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=a4278ba9ff9a0e9f716f76117dba74cb7281c267f2f81f03533e51d1bf6ecb55
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt
모든 라이브러리가 로드되었고 변수가 설정되었습니다.


In [4]:
# -----------------------------------------------------------
# 2. API 기반 메인 파서 (KeyError 해결 포함)
# -----------------------------------------------------------

def process_csv_links_with_api(csv_file='한국근대소설_TEI_XML_작품목록.csv'):
    """ CSV 링크를 읽어 API를 통해 모든 작품의 기본 데이터를 추출합니다. """
    print(f"\n CSV 파일 로드: {csv_file}")
    try: df = pd.read_csv(csv_file)
    except FileNotFoundError: return []
    if '링크' not in df.columns: return []

    valid_links = df['링크'].dropna().astype(str)
    print(f" 총 {len(valid_links)}개의 링크 발견. API 추출 시작...")
    final_data = []

    for index, link_url in tqdm(valid_links.items(), total=len(valid_links), desc="작품별 API 추출"):
        if not link_url.startswith('https://ko.wikisource.org/wiki/'): continue

        page_title = extract_page_title(link_url)
        if not page_title: continue

        raw_content, revision_info = get_wikitext_from_api(page_title)
        if not raw_content: continue

        metadata = extract_metadata(raw_content)
        clean_content = clean_text(raw_content)
        page_url, author_links = generate_urls(page_title, metadata['authors'])

        page_data = {
            'page_id': revision_info.get('revision_id', None), 'title': page_title, 'url': page_url,
            'namespace': 0, 'redirect': None, 'authors': metadata['authors'], 'author_links': author_links,
            'categories': metadata['categories'], 'content': clean_content, 'raw_content': raw_content,
            'composer': metadata['composer'], 'translator': metadata['translator'], 'year': metadata['year'],
            'license': metadata['license'], 'language': metadata['language'],
            'revision_id': revision_info.get('revid'), 'last_modified': revision_info.get('timestamp'),
            'last_contributor': revision_info.get('user'), 'size': len(raw_content) if raw_content else 0,
            'content_size': len(clean_content),
            #  KeyError 해결: 'source' 필드를 여기서 초기화합니다.
            'source': 'API_INITIAL'
        }

        enhanced_data = enhance_with_api(page_data)
        final_data.append(enhanced_data)

        time.sleep(0.3)

    print(f"\n 총 {len(final_data)}개의 작품 데이터 API 추출 완료.")
    return final_data

In [5]:
# -----------------------------------------------------------
# 3. 덤프 수집, 조합, 하이브리드 로직
# -----------------------------------------------------------

def sort_wikisource_parts(page_content_list):
    """ 위키문헌 쪽 문서나 하위 장 제목을 숫자 기반으로 정렬합니다. """
    def extract_key(item):
        title = item['title']
        parts = re.split(r'(\d+)', title)
        key = []
        for part in parts:
            if part.isdigit(): key.append(int(part))
            elif part: key.append(part)
        return key
    return sorted(page_content_list, key=extract_key)

def find_related_dump_pages(dump_file, target_titles):
    """ 덤프 파일에서 목표 작품 제목과 관련된 모든 페이지를 찾습니다. """
    related_pages = {}
    current_titles = set(target_titles)
    TARGET_NAMESPACES = {0, 102}
    print(f"\n 덤프 파일에서 {len(current_titles)}개 작품의 관련 페이지 찾기 시작...")

    try:
        with bz2.open(dump_file, 'rt', encoding='utf-8') as f:
            dump = mwxml.Dump.from_file(f)
            for page in tqdm(dump, desc="덤프 페이지 탐색 중"):
                if page.namespace not in TARGET_NAMESPACES or page.redirect: continue

                text = next(page).text
                is_related = False

                for target_title in current_titles:
                    if page.namespace == 0 and (page.title == target_title or page.title.startswith(target_title + '/') or page.title.startswith(target_title.replace(' ', '_') + '/')):
                        is_related = True; break
                    elif page.namespace == 102:
                        match_base = re.search(r'^(?:[^/]+)', page.title.split(':', 1)[-1])
                        if match_base and (target_title in match_base.group(0).strip() or match_base.group(0).strip() in target_title):
                            is_related = True; break

                if is_related and page.title not in related_pages:
                    related_pages[page.title] = {'title': page.title, 'raw_content': text, 'namespace': page.namespace, 'revisions': [text]}
    except Exception as e:
        print(f"덤프 파싱 중 치명적인 오류 발생: {e}")
    print(f" 덤프에서 총 {len(related_pages)}개의 관련 페이지(본문, 하위 장, 쪽 문서) 수집 완료.")
    return related_pages

def get_content_from_dump(page_title, dump_data_dict, raw_content):
    """ 단일 작품에 대해 덤프 데이터에서 하위 페이지/쪽 문서를 찾아 본문을 조합합니다. (본문 우선 사용) """
    page_content_list = []
    for dump_key, data in dump_data_dict.items():
        if data['title'] == page_title: continue
        is_subpage_or_page = False
        if data['namespace'] == 0 and (data['title'].startswith(page_title + '/') or data['title'].startswith(page_title.replace(' ', '_') + '/')):
            is_subpage_or_page = True
        elif data['namespace'] == 102:
            match_base = re.search(r'^(?:[^/]+)', data['title'].split(':', 1)[-1])
            if match_base and (page_title in match_base.group(0).strip() or match_base.group(0).strip() in page_title):
                is_subpage_or_page = True

        if is_subpage_or_page:
            page_content_list.append({'title': data['title'], 'content': data['revisions'][0], 'namespace': data['namespace']})

    combined_content = ""
    if page_content_list:
        sorted_list = sort_wikisource_parts(page_content_list)
        for item in sorted_list:
            cleaned_page_text = clean_text(item['content'])
            combined_content += cleaned_page_text.strip() + "\n\n"
        final_content = combined_content.strip()
    else:
        final_content = clean_text(raw_content).strip() # 덤프에서 본문을 못 찾으면 메인 페이지 정리본 사용

    return final_content

def process_csv_links_hybrid(csv_file, dump_file, min_content_size=MIN_CONTENT_SIZE_THRESHOLD):
    """ API를 기본으로 사용하고, 본문 크기가 min_content_size 이하인 작품만 덤프로 보강합니다. """
    print("\n--- 1단계: API 기반 기본 추출 및 보강 ---")
    api_results = process_csv_links_with_api(csv_file=csv_file)
    if not api_results: return []

    # 2. 덤프 보강 대상 작품 목록 준비
    dump_target_titles = set()
    for item in api_results:
        # 본문 길이가 min_content_size 이하인 작품만 보강 대상으로 선정
        if item.get('content_size', 0) <= min_content_size:
            dump_target_titles.add(item['title'])

    print(f"\n--- 2단계: 덤프 보강 대상 확인 ---")
    print(f"총 {len(api_results)}개 작품 중, {min_content_size}바이트 이하인 {len(dump_target_titles)}개 작품을 덤프 보강 대상으로 선정했습니다.")

    if not dump_target_titles: return api_results

    # 3. 덤프 파일에서 보강 대상 작품의 모든 관련 페이지 수집
    print(f"\n--- 3단계: 덤프 데이터 수집 (보강 대상만) ---")
    dump_data_dict = find_related_dump_pages(dump_file, dump_target_titles)

    # 4. API 결과에 덤프 본문 적용
    print(f"\n--- 4단계: 덤프 본문 적용 및 최종 데이터 완성 ---")
    final_results = []

    for item in tqdm(api_results, desc="덤프 보강 적용"):
        if item['title'] in dump_target_titles:
            new_content = get_content_from_dump(item['title'], dump_data_dict, item['raw_content'])

            # 덤프 본문이 기존 API 본문보다 길다면 (진짜 본문을 찾았다면) 덮어씁니다.
            if len(new_content) > item.get('content_size', 0):
                item['content'] = new_content
                item['content_size'] = len(new_content)
                item['source'] = 'API_DUMP_HYBRID'
            else:
                item['source'] = 'API_ONLY_SHORT'
        else:
            item['source'] = 'API_ONLY_LONG'

        final_results.append(item)

    print(f"\n총 {len(final_results)}개의 작품 데이터 하이브리드 추출 완료.")
    return final_results

In [8]:
# -----------------------------------------------------------
# 4. XML 출력 함수
# -----------------------------------------------------------

def dataframe_to_xml(df):
    """ Pandas DataFrame을 XML 문자열로 변환합니다. (단락 구조 보존 및 안전한 키 접근) """
    root = ET.Element('works')
    df = df.fillna('')

    for index, row in df.iterrows():
        work = ET.SubElement(root, 'work')

        for col in ['title', 'url', 'year', 'license', 'language']:
            elem = ET.SubElement(work, col)
            elem.text = str(row.get(col, ''))

        authors_elem = ET.SubElement(work, 'authors')
        if isinstance(row.get('authors'), list):
             for author_name in row['authors']:
                author_elem = ET.SubElement(authors_elem, 'author'); author_elem.text = str(author_name)

        categories_elem = ET.SubElement(work, 'categories')
        if isinstance(row.get('categories'), list):
            for cat_name in row['categories']:
                cat_elem = ET.SubElement(categories_elem, 'category'); cat_elem.text = str(cat_name)

        # 콘텐츠 구조화 (@@PARAGRAPH@@ 사용)
        content_text = row.get('content', '')
        content_root = ET.SubElement(work, 'content')

        blocks = content_text.split('@@PARAGRAPH@@')

        for block in blocks:
            block_text = block.strip()
            if not block_text: continue

            if block_text.startswith('='):
                header = ET.SubElement(content_root, 'heading')
                header.text = block_text
            elif block_text.startswith(('*', '-')):
                 list_item = ET.SubElement(content_root, 'list_item')
                 list_item.text = block_text
            else:
                paragraph = ET.SubElement(content_root, 'paragraph')
                paragraph.text = block_text

        work.set('page_id', str(row.get('page_id', '')))
        work.set('revision_id', str(row.get('revision_id', '')))

    xml_string = ET.tostring(root, encoding='utf-8')
    reparsed = minidom.parseString(xml_string)
    return reparsed.toprettyxml(indent="  ", encoding='utf-8')


def output_data(df, base_filename='extracted_wikisource_modern_novels', output_format='json'):
    """ DataFrame을 지정된 형식(json, csv, tsv, xml)으로 저장합니다. """
    output_format = output_format.lower()
    output_filename = f"{base_filename}.{output_format}"

    print(f"\n파일 형식: {output_format.upper()}로 저장 중...")

    try:
        if output_format == 'xml':
            xml_output = dataframe_to_xml(df)
            with open(output_filename, 'wb') as f:
                f.write(xml_output)
        elif output_format == 'json':
            df.to_json(output_filename, orient='records', force_ascii=False, indent=4)
        elif output_format == 'csv':
            df.to_csv(output_filename, index=False, encoding='utf-8')
        elif output_format == 'tsv':
            df.to_csv(output_filename, index=False, sep='\t', encoding='utf-8')
        else:
             print(f"경고: 지원하지 않는 형식 '{output_format}'. JSON으로 저장합니다.")

        print(f" 데이터가 '{output_filename}'으로 저장되었습니다. 다운로드하여 확인하세요.")
    except Exception as e:
        print(f" 데이터 저장 오류 ({output_format}): {e}")
        if output_format == 'xml':
             print("대체: XML 저장에 실패하여 CSV로 저장합니다.")
             df.to_csv(f"{base_filename}_fallback.csv", index=False, encoding='utf-8')


# -----------------------------------------------------------
# 5. 최종 실행 (이 코드를 Colab에 붙여넣고 실행)
# -----------------------------------------------------------

print("--- 하이브리드 파싱 시작 ---")

# min_content_size=50: API로 추출한 본문이 50바이트 이하인 작품만 덤프 보강
extracted_works_hybrid = process_csv_links_hybrid(
    csv_file=csv_file,
    dump_file=dump_file,
    min_content_size=50
)

if extracted_works_hybrid:
    results_df = pd.DataFrame(extracted_works_hybrid)
    print("\n---  최종 하이브리드 데이터 요약 (상위 5개) ---")
    print(results_df[['title', 'authors', 'year', 'content_size', 'source']].head())

    desired_format = 'xml'
    output_data(results_df, output_format=desired_format)
else:
    print("\n 데이터 추출에 실패했습니다.")

--- 하이브리드 파싱 시작 ---

--- 1단계: API 기반 기본 추출 및 보강 ---

 CSV 파일 로드: 한국근대소설_TEI_XML_작품목록.csv
 총 311개의 링크 발견. API 추출 시작...


작품별 API 추출: 100%|██████████| 311/311 [04:33<00:00,  1.14it/s]



 총 311개의 작품 데이터 API 추출 완료.

--- 2단계: 덤프 보강 대상 확인 ---
총 311개 작품 중, 50바이트 이하인 0개 작품을 덤프 보강 대상으로 선정했습니다.

---  최종 하이브리드 데이터 요약 (상위 5개) ---
  title      authors  year  content_size       source
0  혈의 누        [이인직]  1906           389  API_INITIAL
1   철세계  [쥘 베른, 이해조]  None         67624  API_INITIAL
2   자유종        [이해조]  1910           280  API_INITIAL
3  화의 혈        [이해조]  1911         85899  API_INITIAL
4   추월색        [최찬식]  1912         66870  API_INITIAL

파일 형식: XML로 저장 중...
 데이터가 'extracted_wikisource_modern_novels.xml'으로 저장되었습니다. 다운로드하여 확인하세요.


In [10]:
import os
import json
import re
import zipfile
from tqdm import tqdm
import pandas as pd

# -----------------------------------------------------------
# 1. 개별 파일 저장 함수
# -----------------------------------------------------------
def split_and_save_by_work(df, output_dir='individual_works', output_format='xml'):
    """
    DataFrame의 각 행(작품)을 개별 파일로 분리하여 저장합니다.
    (dataframe_to_xml 함수는 메모리에 정의되어 있어야 함)
    """

    # 1. 출력 디렉토리 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"출력 디렉토리 생성: {output_dir}/")
    else:
        print(f"기존 디렉토리 사용: {output_dir}/")

    count = 0

    # 2. 각 작품을 순회하며 저장
    for index, row in tqdm(df.iterrows(), total=len(df), desc=f"개별 작품 ({output_format.upper()}) 저장"):
        try:
            # 파일 이름에 사용할 안전한 제목 생성
            title = str(row['title'])
            # 파일명 금지 문자를 제거하고 공백을 밑줄로 변경
            safe_title = re.sub(r'[\\/:*?"<>|]+', '', title).replace(' ', '_')

            output_filename = os.path.join(output_dir, f"{safe_title}.{output_format}")

            # 현재 작품(행)만 담은 임시 DataFrame 생성
            work_df = pd.DataFrame([row])

            if output_format == 'xml':
                # dataframe_to_xml 함수 사용
                xml_output = dataframe_to_xml(work_df)
                with open(output_filename, 'wb') as f:
                    f.write(xml_output)

            elif output_format == 'json':
                work_df.to_json(output_filename, orient='records', force_ascii=False, indent=4, index=False)

            elif output_format == 'csv':
                work_df.to_csv(output_filename, index=False, encoding='utf-8')

            else:
                print(f"경고: 지원하지 않는 형식 '{output_format}'입니다. 저장을 건너뜝니다.")
                break

            count += 1

        except Exception as e:
            print(f"\n{title} 작품 저장 중 오류 발생: {e}")
            continue

    print(f"\n총 {count}개 작품을 {output_format.upper()} 형식으로 {output_dir}/ 에 저장 완료.")
    return output_dir # 저장된 디렉토리 이름 반환


def zip_directory(dir_name, zip_name):
    """지정된 디렉토리를 ZIP 파일로 압축합니다."""
    print(f"\n디렉토리 '{dir_name}'을 ZIP 파일 '{zip_name}'으로 압축 중...")
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(dir_name):
            for file in files:
                file_path = os.path.join(root, file)
                # ZIP 파일 내 경로를 dir_name을 기준으로 상대 경로로 설정
                zipf.write(file_path, os.path.relpath(file_path, os.path.dirname(dir_name)))
    print(f"압축 완료: {zip_name}")


# -----------------------------------------------------------
# 2. 실행 및 ZIP 압축 (Colab용)
# -----------------------------------------------------------

# 1. 분리할 DataFrame과 원하는 형식을 설정
# (이전 셀에서 생성된 results_df 변수 사용)
if 'results_df' in locals():
    # 원하는 출력 형식을 여기서 선택하세요: 'xml', 'json', 'csv'
    SPLIT_FORMAT = 'xml'
    OUTPUT_DIR = f'works_split_{SPLIT_FORMAT}'
    ZIP_FILENAME = f'{OUTPUT_DIR}.zip'

    # 2. 개별 파일 저장 실행
    saved_dir = split_and_save_by_work(
        df=results_df,
        output_dir=OUTPUT_DIR,
        output_format=SPLIT_FORMAT
    )

    # 3. ZIP 압축 실행
    zip_directory(saved_dir, ZIP_FILENAME)

    # 4. Colab 환경에서 다운로드 링크 생성 (실제 다운로드를 위한 명령어)
    from google.colab import files
    print(f"\n다운로드 링크 생성 중...")
    # files.download(ZIP_FILENAME) # Colab 환경에서 다운로드 실행
    print(f"다운로드를 위해 'files.download(\"{ZIP_FILENAME}\")' 명령어를 실행하세요.")

else:
    print("오류: 'results_df' DataFrame이 메모리에 없습니다. 이전 추출 셀을 실행했는지 확인하세요.")

기존 디렉토리 사용: works_split_xml/


개별 작품 (XML) 저장: 100%|██████████| 311/311 [00:01<00:00, 177.69it/s]



총 311개 작품을 XML 형식으로 works_split_xml/ 에 저장 완료.

디렉토리 'works_split_xml'을 ZIP 파일 'works_split_xml.zip'으로 압축 중...
압축 완료: works_split_xml.zip

다운로드 링크 생성 중...
다운로드를 위해 'files.download("works_split_xml.zip")' 명령어를 실행하세요.
